# Data Translation and Reorganization

## Imports

In [ ]:
import pandas as pd
import numpy as np
from google_trans_new import google_translator

## Loading datasets

In [ ]:
validData = pd.read_csv("valid.csv")
testData = pd.read_csv("test.csv")
trainData = pd.read_csv("train.csv")

## obtaining the relevant columns from the data

In [ ]:
filtered_trainData=trainData.loc[:,["context","prompt","utterance"]]
filtered_validData=validData.loc[:,["context","prompt","utterance"]]
filtered_testData=testData.loc[:,["context","prompt","utterance"]]

## Creating placeholders for ordered Data

In [ ]:
emptyData=np.zeros((filtered_testData.shape))
orderedTest_df = pd.DataFrame(emptyData,columns = ['emotion', 'context', 'response'])
emptyData=np.zeros((filtered_validData.shape))
orderedValid_df = pd.DataFrame(emptyData,columns = ['emotion', 'context', 'response'])
emptyData=np.zeros((filtered_trainData.shape))
orderedTrain_df = pd.DataFrame(emptyData,columns = ['emotion', 'context', 'response'])

## Splitting text to emotion,context,response

### Test

In [ ]:
for i in range(0,filtered_testData.shape[0],2):
    if filtered_testData["prompt"][i]==filtered_testData["prompt"][(i+1)%filtered_testData.shape[0]]:
        orderedTest_df['emotion'][i/2]=filtered_testData['context'][i]
        orderedTest_df['context'][i/2]=filtered_testData['utterance'][i]
        orderedTest_df['response'][i/2]=filtered_testData['utterance'][i+1]

In [ ]:
orderedTest_df=orderedTest_df[(orderedTest_df["emotion"]!=0)]

### Valid

In [ ]:
for i in range(0,filtered_validData.shape[0],2):
    if filtered_validData["prompt"][i]==filtered_validData["prompt"][(i+1)%filtered_validData[0]]:
        orderedValid_df['emotion'][i/2]=filtered_validData['context'][i]
        orderedValid_df['context'][i/2]=filtered_validData['utterance'][i]
        orderedValid_df['response'][i/2]=filtered_validData['utterance'][i+1]

In [ ]:
orderedValid_df=orderedValid_df[(orderedValid_df["emotion"]!=0)]

### Train

In [ ]:
for i in range(0,filtered_trainData.shape[0],2):
    if filtered_trainData["prompt"][i]==filtered_trainData["prompt"][(i+1)%filtered_trainData[0]]:
        orderedTrain_df['emotion'][i/2]=filtered_trainData['context'][i]
        orderedTrain_df['context'][i/2]=filtered_trainData['utterance'][i]
        orderedTrain_df['response'][i/2]=filtered_trainData['utterance'][i+1]

In [ ]:
orderedTrain_df=orderedTrain_df[(orderedTrain_df["emotion"]!=0)]

### Saving Organized Dataset

In [ ]:
orderedTrain_df.to_csv('TrainOrganized.csv', index = False)
orderedTest_df.to_csv('TestOrganized.csv', index = False)
orderedValid_df.to_csv('ValidOrganized.csv', index = False)

## Translating

### reading dataset

In [ ]:
orderedTrain_df=pd.read_csv('TrainOrganized.csv')
orderedTest_df=pd.read_csv('TestOrganized.csv')
orderedValid_df=pd.read_csv('ValidOrganized.csv')

### Obtaining parts in need of translation

In [ ]:
train_to_translate=orderedTrain_df.loc[:,["context","response"]]
valid_to_translate=orderedValid_df.loc[:,["context","response"]]
test_to_translate=orderedTest_df.loc[:,["context","response"]]

### placeholders for translated data

In [ ]:
translatedTrain=train_to_translate.copy()
translatedTrain.reset_index(drop=True, inplace=True)
train_to_translate.reset_index(drop=True, inplace=True)

translatedValid=valid_to_translate.copy()
translatedValid.reset_index(drop=True, inplace=True)
valid_to_translate.reset_index(drop=True, inplace=True)

translatedTest=test_to_translate.copy()
translatedTest.reset_index(drop=True, inplace=True)
test_to_translate.reset_index(drop=True, inplace=True)

### Initializing translator 

In [ ]:
translator = google_translator(url_suffix="eg", timeout=500)  

### Translating test

In [ ]:
for i in range(0,test_to_translate.shape[0]):
    translate_text = translator.translate(test_to_translate["context"][i],lang_tgt='ar')
    translatedTest["context"][i]=translate_text
    translate_text = translator.translate(test_to_translate["response"][i],lang_tgt='ar')
    translatedTest["response"][i]=translate_text
    if(i%500==0):
        print(i)
        print(translatedTest["context"][i])
        print(translatedTest["response"][i])
        translatedTest.to_csv('TranslatedTest.csv', encoding="utf-8-sig", index = False)
translatedTest.to_csv('TranslatedTest.csv', encoding="utf-8-sig", index = False)

### Translating Valid

In [ ]:
for i in range(0,valid_to_translate.shape[0]):
    translate_text = translator.translate(valid_to_translate["context"][i],lang_tgt='ar')
    translatedValid["context"][i]=translate_text
    translate_text = translator.translate(valid_to_translate["response"][i],lang_tgt='ar')
    translatedValid["response"][i]=translate_text
    if(i%500==0):
        print(i)
        print(translatedValid["context"][i])
        print(translatedValid["response"][i])
        translatedValid.to_csv('TranslatedValid.csv', encoding="utf-8-sig", index = False)
translatedValid.to_csv('TranslatedValid.csv', encoding="utf-8-sig", index = False)

### Translating Train

In [ ]:
for i in range(0,train_to_translate.shape[0]):
    translate_text = translator.translate(train_to_translate["context"][i],lang_tgt='ar')
    translatedTrain["context"][i]=translate_text
    translate_text = translator.translate(train_to_translate["response"][i],lang_tgt='ar')
    translatedTrain["response"][i]=translate_text
    if(i%500==0):
        print(i)
        print(translatedTrain["context"][i])
        print(translatedTrain["response"][i])
        translatedTrain.to_csv('TranslatedTrain.csv', encoding="utf-8-sig", index = False)
translatedTrain.to_csv('TranslatedValid.csv', encoding="utf-8-sig", index = False)

### Adding Emotion column

In [ ]:
translatedTrain["emotion"]=orderedTrain["emotion"]
translatedTrain=translatedTrain[["emotion","context","response"]]

translatedTest["emotion"]=orderedTest["emotion"]
translatedTest=translatedTest[["emotion","context","response"]]

translatedValid["emotion"]=orderedValid["emotion"]
translatedValid=translatedValid[["emotion","context","response"]]

In [ ]:
translatedTrain.to_csv('trainFinal.csv', encoding="utf-8-sig", index = False)
translatedTest.to_csv('testFinal.csv', encoding="utf-8-sig", index = False)
translatedValid.to_csv('validFinal.csv', encoding="utf-8-sig", index = False)